In [1]:
using Plots
using LinearAlgebra



# TURBINE

In [2]:
using Symbolics
@variables T1 rp cp cv η Nt kcoeff eta_c
T2s = T1/(rp^kcoeff)
T2a = T1 - η * (T1-T2s)
@show simplify(T2a / T1)


simplify(T2a / T1) = (η + rp^kcoeff - η*(rp^kcoeff)) / (rp^kcoeff)


(η + rp^kcoeff - η*(rp^kcoeff)) / (rp^kcoeff)

In [3]:

T2sc = T1*rp^kcoeff;
T2ac = T1+(T2sc-T1)/η
@show simplify(T2ac/T1)


simplify(T2ac / T1) = (η + rp^kcoeff - 1.0) / η


(η + rp^kcoeff - 1.0) / η

In [24]:
using Symbolics
# cycle assumptions
@variables Tci cp_cycle Pdiv Pblk cycle_mflow
#blanket variables
@variables Tmax_blk cp_blk ac_blk hx_eff_blk X 
@variables Tmax_div cp_div ac_div hx_eff_div Y

print_for_copy = true


blk_ΔT_bhx  = (1/X)*(Tmax_blk - Tci) * hx_eff_blk
cyc_ΔT_bhx  = (Tmax_blk-Tci) * hx_eff_blk
Tmin_blk    = Tmax_blk - blk_ΔT_bhx
Tin_blk     = Tmin_blk*ac_blk
q_fus_blk   = cp_blk * (Tmax_blk - Tin_blk)
pump_blk    = cp_blk * (Tin_blk-Tmin_blk)
q_del_blk   = cp_cycle * cyc_ΔT_bhx             # cycle sidew
qd_blk      = cp_blk   * blk_ΔT_bhx             # blanket side

Tci_d       = Tci + cyc_ΔT_bhx
div_ΔT_dhx  = (1/Y)*(Tmax_div - Tci_d) * hx_eff_div
cyc_ΔT_dhx  = (Tmax_div - Tci_d) * hx_eff_div
Tmin_div    = Tmax_div - div_ΔT_dhx
Tin_div     = Tmin_div * ac_div
pump_div    = cp_div *(Tin_div-Tmin_div)
q_fus_div   = cp_div * (Tmax_div - Tin_div)
q_del_div   = cp_cycle * cyc_ΔT_dhx
qd_div      = cp_div * div_ΔT_dhx

mflow_b = cycle_mflow * X
mflow_d = cycle_mflow * Y 
# blk_expr = Symbolics.solve_for([q_del_blk * cycle_mflow - qd_blk * mflow_b ~ 0 , q_del_div * cycle_mflow - qd_div * mflow_d ~ 0],[X, Y])
mflow_div = Pdiv/q_fus_div
mflow_blk = Pblk/q_fus_blk
expr_blk = Symbolics.solve_for([cycle_mflow * X - mflow_blk ~ 0],cycle_mflow)
expr_div = Symbolics.solve_for([cycle_mflow * Y - mflow_div ~ 0],cycle_mflow)

Tco_cycle_div = Tci_d + cyc_ΔT_dhx

if print_for_copy
    @show expand(expand(Tmin_blk))
    @show q_fus_blk
    @show pump_blk
    @show q_del_blk
    @show qd_blk

    @show Tci_d

    @show Tmin_div
    @show q_fus_div
    @show pump_div
    @show q_del_div
    @show qd_div
    @show Tco_cycle_div
    @show mflow_blk
    @show mflow_div

    @show expr_blk
    @show expr_div
end


expand(expand(Tmin_blk)) = Tmax_blk + (Tci*hx_eff_blk - Tmax_blk*hx_eff_blk) / X
q_fus_blk = cp_blk*(Tmax_blk - ac_blk*(Tmax_blk + (-hx_eff_blk*(Tmax_blk - Tci)) / X))
pump_blk = cp_blk*(ac_blk*(Tmax_blk + (-hx_eff_blk*(Tmax_blk - Tci)) / X) + (hx_eff_blk*(Tmax_blk - Tci)) / X - Tmax_blk)
q_del_blk = cp_cycle*hx_eff_blk*(Tmax_blk - Tci)
qd_blk = (cp_blk*hx_eff_blk*(Tmax_blk - Tci)) / X
Tci_d = Tci + hx_eff_blk*(Tmax_blk - Tci)
Tmin_div = Tmax_div + (-hx_eff_div*(Tmax_div - Tci - hx_eff_blk*(Tmax_blk - Tci))) / Y
q_fus_div = cp_div*(Tmax_div - ac_div*(Tmax_div + (-hx_eff_div*(Tmax_div - Tci - hx_eff_blk*(Tmax_blk - Tci))) / Y))
pump_div = cp_div*(ac_div*(Tmax_div + (-hx_eff_div*(Tmax_div - Tci - hx_eff_blk*(Tmax_blk - Tci))) / Y) + (hx_eff_div*(Tmax_div - Tci - hx_eff_blk*(Tmax_blk - Tci))) / Y - Tmax_div)
q_del_div = cp_cycle*hx_eff_div*(Tmax_div - Tci - hx_eff_blk*(Tmax_blk - Tci))
qd_div = (cp_div*hx_eff_div*(Tmax_div - Tci - hx_eff_blk*(Tmax_blk - Tci))) / Y
Tco_cycle_div = Tci + hx

(Pdiv / (cp_div*(Tmax_div - ac_div*(Tmax_div + (-hx_eff_div*(Tmax_div - Tci - hx_eff_blk*(Tmax_blk - Tci))) / Y)))) / Y

In [23]:
using Symbolics
# regen and turbine
@variables eff_regen Tmax_cycle Trci Trco Trho Trhi aturb cp_cycle cycle_mflow
#breeder props
@variables Tmax_br cp_br hx_eff_br Z v_br DP_br eta_pump P_br

print_for_copy = true

# Z = mcp_breeder / mcp_cycle

#preliminaries
win_br          = DP_br * v_br / eta_pump;
ΔT_pump_br      = (win_br - v_br * DP_br)/cp_br;


Trhi = aturb * Tmax_cycle       #turbine outlet
# Trci = Tco_cycle_div          #after divertor hx

ΔT_regen = eff_regen * (Trhi - Trci)    #temp chance
Trco    = Trci + ΔT_regen               # regen outlet temp
Trho    = Trhi - ΔT_regen
@variables C_br C_cyc
# C_cyc   = mflow_cycle * cp_cycle
# C_br    = mflow_breeder * cp_br
Cmin = min(C_br,C_cyc)

display("if Z > 1")
# ΔT_hx_br    = Cmin/C_br  * (Tmax_br - Trco) * hx_eff_br
# ΔT_hx_cyc   = Cmin/C_cyc * (Tmax_br - Trco) * hx_eff_br
ΔT_hx_br    = (1/Z)  * (Tmax_br - Trco) * hx_eff_br
ΔT_hx_cyc   = (Tmax_br - Trco) * hx_eff_br

Tmax_cycle  = Trco  + ΔT_hx_cyc
Tmin_br     = Tmax_br - ΔT_hx_br
Tin_br      = Tmin_br + ΔT_pump_br
q_fus_br    = cp_br * (Tmax_br - Tin_br)
mflow_br    = P_br/q_fus_br
expr_br    = Symbolics.solve_for([cycle_mflow * cp_cycle * Z - mflow_br * cp_br ~ 0],cycle_mflow)
xy_from_breeder = Symbolics.solve_for([expr_br - expr_blk ~ 0, expr_br ~ expr_div = 0],[X, Y])



if print_for_copy
    println("Trhi     = @(Z) $(Trhi)")
    println("Trco     = @(Z) $(Trco)")
    println("Trho     = @(Z) $(Trho)")
    println("Tmax_cycle     = @(Z) $(Tmax_cycle)")
    println("Tmin_br  = @(Z) $(Tmin_br)")
    println("Tin_br   = @(Z) $(Tin_br)")
    println("q_fus_br = @(Z) $(q_fus_br)")
    println("mflow_br = @(Z) $(mflow_br)")
    println("expr_br  = @(Z) $(expr_br)")
end

display("if Z < 1")
ΔT_hx_br    = (Tmax_br - Trco) * hx_eff_br
ΔT_hx_cyc   = Z*(Tmax_br - Trco) * hx_eff_br

Tmax_cycle  = Trco  + ΔT_hx_cyc
Tmin_br     = Tmax_br - ΔT_hx_br
Tin_br      = Tmin_br + ΔT_pump_br
q_fus_br    = cp_br * (Tmax_br - Tin_br)
mflow_br    = P_br/q_fus_br
expr_br    = Symbolics.solve_for([cycle_mflow * cp_cycle * Z - mflow_br * cp_br ~ 0],cycle_mflow)

if print_for_copy
    println("Trhi     = @(Z) $(Trhi)")
    println("Trco     = @(Z) $(Trco)")
    println("Trho     = @(Z) $(Trho)")
    println("Tmax_cycle = @(Z) $(Tmax_cycle)")
    println("Tmin_br  = @(Z) $(Tmin_br)")
    println("Tin_br   = @(Z) $(Tin_br)")
    println("q_fus_br = @(Z) $(q_fus_br)")
    println("mflow_br = @(Z) $(mflow_br)")
    println("expr_br  = @(Z) $(expr_br)")
end

"if Z > 1"

LoadError: AssertionError: islinear